# Importing libraries

In [2]:
import pandas as pd 
import numpy as np 

In [3]:
import os, sys, glob, shutil
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/Users/shubham/work/hurricane_damage_prediction'

In [19]:
from geopy.geocoders import Nominatim as osm_geocoder

In [52]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(osm_geocoder.geocode, min_delay_seconds=1)

# Loading and processing data

In [4]:
address = pd.read_csv("data/addresses_and_losses.csv")

/Users/shubham/anaconda3/envs/hurricane/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (43,46,47,50,55,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
address.head()

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Business_Address,Business_City,Business_State,Business_Zip,Disaster_Id,...,FF_UP52_V,LEI_UP53_V,VEH_UP54_V,VES_UP55_V,ACF_UP56_V,RR_UP58_V,RM_UP59_V,WC_UP60_V,WC_UP61_V,NP_UP62_V
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Business,NaN,NaN,NaN,NaN,314 CUT OFF RD,PORT ARANSAS,TX,78373.0,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Household,622 JAMES ST,REFUGIO,TX,78377.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Household,RV PARK AVE # 44,CORPUS CHRISTI,TX,78401.0,NaN,NaN,NaN,NaN,1008079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
damage_data = address[["applicant_type", "Individual_Address", "Individual_City", "Individual_State", "Individual_Zip", "Orig_RE_Verified_Loss"]]

Removing rows which consists of NA values. Considering NA values to be "not recorded" for now. Further analysis might be needed to check if NA Loss means 0 loss.

In [44]:
damage_data.dropna(subset=['Individual_Address', 'Orig_RE_Verified_Loss'], inplace=True)

/Users/shubham/anaconda3/envs/hurricane/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
damage_data.head()

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Orig_RE_Verified_Loss
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,58999.0
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,2563.0
7,Household,1202 MESQUITE ST SPACE 546,FULTON,TX,78358.0,26823.0
9,Household,314 DRISCOLL ST,WOODSBORO,TX,78393.0,99357.0
13,Household,3301 MEADOWLANE ST,VICTORIA,TX,77901.0,55813.0


In [55]:
osm = osm_geocoder(user_agent="hurricane")

In [104]:
loc = osm.geocode("1202 MESQUITE ST SPACE 546, FULTON, TX")

In [106]:
loc

In [102]:
loc.address

'Meadowlane Street, Angleton, Brazoria County, Texas, 77515, United States of America'

In [103]:
loc.point

Point(29.181736, -95.421223, 0.0)

In [111]:
count = 0
def geocode(x):
    global count
    count+=1
    if count % 100 == 0:
        print(count)
    str_address = x.Individual_Address + ", " + x.Individual_City + ", " + x.Individual_State + ", " + str(int(x.Individual_Zip))
    print(str_address)
    if x.coordinates == None:
        try:
            loc = osm.geocode(str_address, timeout=5)
            return loc.point
        except:
            return None
    else:
        return x.coordinates

In [112]:
damage_data['coordinates'] = damage_data.apply(lambda x: geocode(x), axis=1)

314 CUT OFF ROAD, PORT ARANSAS, TX, 78373
5901 HORIZON DRIVE, CORPUS CHRISTI, TX, 78408
1202 MESQUITE ST SPACE 546, FULTON, TX, 78358
314 DRISCOLL ST, WOODSBORO, TX, 78393
3301 MEADOWLANE ST, VICTORIA, TX, 77901
189 PRESA ST, BEEVILLE, TX, 78102
7403 SEMINOLE ST, BAYTOWN, TX, 77521
291 COUNTY ROAD   # 4801, DAYTON, TX, 77535
3060 LENORE AVE, INGLESIDE, TX, 78362
708 RHINEGOLD ST, VICTORIA, TX, 77901
11447 SAGEWHITE DR, HOUSTON, TX, 77089
8414 LAURA KOPPE RD, HOUSTON, TX, 77028
413 TROJAN ST LOT 9, PORT ARANSAS, TX, 78373
3714 AVE E 1/2, SANTA FE, TX, 77510
1207 SIGMA ST, PASADENA, TX, 77504
12115 PALMTON ST, HOUSTON, TX, 77034
4851 HIGHWAY 35 N # 14, ROCKPORT, TX, 78382
9602 ROSEHAVEN DRIVE, HOUSTON, TX, 77051
4422 SAFFRON LN, FRIENDSWOOD, TX, 77546
210 OAK ST, ROCKPORT, TX, 78382
4122 SOLWAY LN, HOUSTON, TX, 77025
8130 77 A COUNTY ROAD 3501 # A, TAFT, TX, 78390
5727 CERRITOS DR, HOUSTON, TX, 77035
108 PRAIRIE WILDE ST, FRIENDSWOOD, TX, 77546
6931 GUHN RD LOT 15, HOUSTON, TX, 77040
957

2512 RIVER RIDGE DR, CONROE, TX, 77385
20703 WINLOCK TRACE DR, KATY, TX, 77450
5205 CALVACADE, HOUSTON, TX, 77026
4420 LEMAC DR, HOUSTON, TX, 77096
6286 PAT AVE., PORT ARTHUR, TX, 77640
5210 BARRETT STREET, ORANGE, TX, 77630
13590 INWOOD DR, BEAUMONT, TX, 77713
1604 W MARTHA LN, PASADENA, TX, 77502
2310 RYANN LN, GROVES, TX, 77619
20398 LANA LN, PORTER, TX, 77365
306 THIRD ST, BAYSIDE, TX, 78340
12311 NORKEY ST, HOUSTON, TX, 77086
2730 MAGNOLIA LN., GROVES, TX, 77619
11711 GREENMESA DR, HOUSTON, TX, 77044
3758 GRENNOCH LN, HOUSTON, TX, 77025
13902 BRIARHILLS PKWY, HOUSTON, TX, 77077
5307 GOLD MEDAL CIR, HOUSTON, TX, 77041
16120 STUEBNER AIRLINE RD APT 504, SPRING, TX, 77379
835 MEADOWLARK ST, BRIDGE CITY, TX, 77611
28015 JILLIAN OAKS LN, SPRING, TX, 77386
235 N AUSTIN DR, HOLIDAY LAKES, TX, 77515
401 PINEY WOODS ROAD, SHEPHERD, TX, 77371
11107 WINDMARK PL, HOUSTON, TX, 77099
8406 GOLD CREEK DR, HOUSTON, TX, 77080
15902 RED WILLOW DR, HOUSTON, TX, 77084
6507 GRAND CANYON GATE DR, KATY, 

2214 JACKSON ST, PORT LAVACA, TX, 77979
11226 TAMWORTH DR, HOUSTON, TX, 77016
20718 LIVE OAK RD, CROSBY, TX, 77532
3875 BRANDON ST, BEAUMONT, TX, 77705
2876 SOLOMON ST, PORT ARTHUR, TX, 77640
2305 MCNAY DR, PASADENA, TX, 77506
11919 PONDWOOD DR, CYPRESS, TX, 77429
5221 5TH ST, PORT ARTHUR, TX, 77642
511 ATLANTA AVE, NEDERLAND, TX, 77627
508 AVENUE B, SINTON, TX, 78387
3853 COUNTY ROAD 651, BRAZORIA, TX, 77422
3634 CURTISS ST, CORPUS CHRISTI, TX, 78405
3307 MONTROSE ST, ORANGE, TX, 77632
11315 ASHWOOD DR, HUMBLE, TX, 77338
10911 CRANBROOK RD, HOUSTON, TX, 77042
19126 JADEN MCCADE CT, PORTER, TX, 77365
1910 12TH ST, PORT ARTHUR, TX, 77640
512 N 36TH ST, NEDERLAND, TX, 77627
4139 SOUTH ACRES DR, HOUSTON, TX, 77047
25782 PEACH DR, NEW CANEY, TX, 77357
1046 DOLPHIN PL, CORPUS CHRISTI, TX, 78411
1251 TRACE DR, HOUSTON, TX, 77077
9226 CARMALEE ST, HOUSTON, TX, 77075
15514 TUMBLING RAPIDS DRIVE, HOUSTON, TX, 77084
21302 MYSTIC OAK DR, CYPRESS, TX, 77433
11622 SILENT ELM ST, HOUSTON, TX, 77044


123 TIMBERCREEK LN, LUMBERTON, TX, 77657
14530 CLAYCROFT CT, CYPRESS, TX, 77429
6606 HARTWICK RD, HOUSTON, TX, 77016
502 STARLIGHT, INGLESIDE, TX, 78362
863 SHARP ST, BRIDGE CITY, TX, 77611
4334 PERIDOT LN, FRIENDSWOOD, TX, 77546
100 LAKEVIEW DR, LAKE CITY, TX, 78368
14010 GREEN TEAL LN, HOUSTON TX, TX, 77039
210 N GARDEN AVE, NIXON, TX, 78140
23803 AYSCOUGH LN, KATY, TX, 77493
1464 S 19TH ST, CORPUS CHRISTI, TX, 78404
11918 CORAL REEF DR, HOUSTON, TX, 77044
630 ARVANA ST, HOUSTON, TX, 77034
1815 PEWTER CT, KATY, TX, 77493
2510 CLOYDE STREET, CORPUS CHRISTI, TX, 78404
12313 KAYLA LN, HOUSTON, TX, 77015
4725 DONEGAL DR, CORPUS CHRISTI, TX, 78413
16303 CENTRE COURT PL, SPRING, TX, 77379
9501 BLUE BONNET DR, TEXAS CITY, TX, 77591
330 HIGHWAY 332 W, LAKE JACKSON, TX, 77566
4413 BONNER DR, CORPUS CHRISTI, TX, 78411
1401 SMOKEHOUSE RD LOT M2, ROCKPORT, TX, 78382
4311 LAFAYETTE ST, BELLAIRE, TX, 77401
12621 SANDPIPER RD, ANGLETON, TX, 77515
2130 BEACON LIGHT LN, FRESNO, TX, 77545
14857 COUNTY

1495 WOOTEN RD, BEAUMONT, TX, 77707
18933 ENGLIN RD, WINNIE, TX, 77665
2714 WOODLAWN ST, DICKINSON, TX, 77539
502 WEST MAHAN, PORT LAVACA, TX, 77979
3810 CYPRESS POINT DR, BEAUMONT, TX, 77707
847 JUDIWAY ST, HOUSTON, TX, 77018
11810 E MORGAN DR, HOUSTON, TX, 77065
288 MISSION BELL RD, VICTORIA, TX, 77905
20294 SPRUCE DR, MONTGOMERY, TX, 77356
4813 COSNER DR, CORPUS CHRISTI, TX, 78415
12522 GOTHAM DRIVE, HOUSTON, TX, 77089
4704 DAIRY OAKS RD, CROSBY, TX, 77532
20627 CRANFIELD DR, KATY, TX, 77450
205 W MARVICK, PASADENA, TX, 77506
6302 GASTON STREET, HOUSTON, TX, 77016
4015 S MAIN STREET, VIDOR, TX, 77662
4503 MAGNOLIA PINA, PEARLAND, TX, 77584
5813 HAMPSHIRE ROAD, CORPUS CHRISTI, TX, 78408
738 COBIA DR, SURFSIDE BEACH, TX, 77541
10007 WAVING FIELD DR, HOUSTON, TX, 77064
223 W. ESCOBEDO ST., TAFT, TX, 78390
1135 N TANNAHILL STREET, VIDOR, TX, 77662
617 NOLAN ST, NAVASOTA, TX, 77868
314 DIERS LANE, LA GRANGE, TX, 78945
399 FRENCH RD, LUMBERTON, TX, 77657
13 STRADFORD DRIVE, ORANGE, TX, 77

...

5627 RUTHERGLENN DRIVE, HOUSTON, TX, 77096
20065 RIVER RIDGE DRIVE, PORTER, TX, 77365
52900
5235 KNIGHTWOOD ST, HOUSTON, TX, 77016
2604 SOLERO RIDGE LN, HOUSTON, TX, 77089
25818 CELTIC TERRACE DRIVE, KATY, TX, 77494
15727 FOUR LEAF DR, HOUSTON, TX, 77084
1815 HAVER STREET, HOUSTON, TX, 77006
8830 WOODLYN RD, HOUSTON, TX, 77078
3213 TIMBERLANE CIR, ORANGE, TX, 77632
4609 MORNING DOVE LN, ALVIN, TX, 77511
602 PARK LANE, FRIENDSWOOD, TX, 77546
12702 ARCHWOOD LANE, CYPRESS, TX, 77429
5925 LONG CREEK, CORPUS CHRISTI, TX, 78414
1145 COUNTY ROAD 136A, ALVIN, TX, 77511
620 DOLORES ST, TAFT, TX, 78390
3525 CHARLES AVE, PEARLAND, TX, 77584
4131 MAGNOLIA CIR, WHARTON, TX, 77488
800 MARINERS DR SLIP 1707, KEMAH, TX, 77565
158 DERNAL DR, VICTORIA, TX, 77905
1417 HOLLOMAN DR, PORT LAVACA, TX, 77979
20331 REDBUD DR., NEW CANEY, TX, 77357
21871 CALVIN RD, HUFFMAN, TX, 77336
4902 VINTAGE GROVE CT, KATY, TX, 77449
4422 HWY 44, ROBSTOWN, TX, 78380
17811 ALPINE BROOK LN, ATASCOCITA, TX, 77346
1218 GLENCRE

1111 GRAHAM RD, CORPUS CHRISTI, TX, 78418
2128 GREENLEE LANE, DICKINSON, TX, 77539
5730 RIDGEWAY DRIVE, HOUSTON, TX, 77033
4719 MONTCLAIR HILL LN, FRESNO, TX, 77545
826 FM 1406 RD, WINNIE, TX, 77665
10418 CARIBOU COVE, MISSOURI CITY, TX, 77459
438 LEVIEN ROAD, WOODSBORO, TX, 78393
6414 SCOTT RD, PEARLAND, TX, 77581
103 HANCOCK, ODEM, TX, 78370
1414 AIRLINE N, ROSHARON, TX, 77583
1018 RED ROCK CANYON DRIVE, KATY, TX, 77450
7604 BARRYKNOLL LANE, PORT ARTHUR, TX, 77642
5320 CIMARRON DR, DICKINSON, TX, 77539
862 SOUTH BAY STREET, ARANSAS PASS, TX, 78336
15610 THORNBROOK DR, HOUSTON, TX, 77084
7231 BRANFORD PARK LN, RICHMOND, TX, 77407
19132 LORDS ST, NEW CANEY, TX, 77357
8934 WOODLYN, HOUSTON, TX, 77078
1406 LAKEVIEW DRIVE, SOUR LAKE, TX, 77659
14606 MORINSCOTT DR, HOUSTON, TX, 77049
4101 WALNUT HILLS DR, CORPUS CHRISTI, TX, 78413
3227 RIPPLING FALLS LN, DICKINSON, TX, 77539
602 REGAL ST, HOUSTON, TX, 77034
2160 HOLDEN RD, ARANSAS PASS, TX, 78336
165 E. SUSAN CIRCLE, ORANGE, TX, 77630
2248

606 TURNEY, HOUSTON, TX, 77037
1914 HUDSON ST, CORPUS CHRISTI, TX, 78416
3385 HIGHWAY 327 WEST, SILSBEE, TX, 77656
7807 SHELBURNE, SPRING, TX, 77379
412 NAVAJO DR, VICTORIA, TX, 77904
11584 DAVIDSON RD, BEAUMONT, TX, 77705
5572 COUNTY ROAD 2047, ODEM, TX, 78370
725 TOLAND, TAFT, TX, 78390
6631 39TH STREET, GROVES, TX, 77619
4230 HOWCHER STREET, HOUSTON, TX, 77047
1821 BIRCH, TEXAS CITY, TX, 77591
9535 BARR SPRING DR, HUMBLE, TX, 77396
4953 KATHY, CORPUS CHRISTI, TX, 78411
3503 GLENN LAKES LANE, MISSOURI CITY, TX, 77459
1119 WEST 21ST STREET, HOUSTON, TX, 77008
3334 FM 359, RICHMOND, TX, 77469
5017 CRESTWICK DRIVE, CORPUS CHRISTI, TX, 78413
2075 LIBERTY AVE, BEAUMONT, TX, 77701
3300 CR 190, ROSHARON, TX, 77583
3038 CARVER DR, CORPUS CHRISTI, TX, 78405
8025 PRIVATE ROAD 4405, TAFT, TX, 78390
5821 LUCY LN, PASADENA, TX, 77505
2940 MARION, VIDOR, TX, 77662
4642 ANICE ST., HOUSTON, TX, 77039
6501 BRISK WIND DR., CORPUS CHRISTI, TX, 78414
24107 PEPPERRELL PLACE, KATY, TX, 77493
1618 N. PLUM 

2812 DREW ST., HOUSTON, TX, 77004
5608 WOODLAND DR, ORANGE, TX, 77632
19738 TWIN CANYON CT, KATY, TX, 77450
22723 JAY DRIVE, SPRING, TX, 77373
710 E. CORNELIA CIR, CORPUS CHRISTI, TX, 78408
442 PENNINGTON DR, CORPUS CHRISTI, TX, 78412
3 NEPTUNE DR, ROCKPORT, TX, 78382
12438 LEDGER LN., HOUSTON, TX, 77015
6760 39TH ST, GROVES, TX, 77619
2922 SILVER MAPLE CT, FRIENDSWOOD, TX, 77546
9807 RIBBONWOOD ST, HOUSTON, TX, 77078
7242 GRANTS HOLLOW LN, RICHMOND, TX, 77407
831 COLORADO AVE, PORT ARTHUR, TX, 77642
10819 SAGELEAF LN., HOUSTON, TX, 77089
544 LYDIA ANNE CHANNEL, PORT ARANSAS, TX, 78373
306 INDIAN BAYOU, HOUSTON, TX, 77057
15407 PARKVIEW ST, HOUSTON, TX, 77071
615 S 4TH, NEDERLAND, TX, 77627
3427 CEDAR HILL LN, HOUSTON, TX, 77093
405 1/2 E VERBINA ST, TAFT, TX, 78390
405 E VERBINA, TAFT, TX, 78390
504 NORTH TEXAS AVE, ANAHUAC, TX, 77514
58 E HARDING CIR, ORANGE, TX, 77630
7826 CR 128, ALVIN, TX, 77511
14010 GOODRIDGE DR, HOUSTON, TX, 77048
4802 MAJESTIC, HOUSTON, TX, 77026
7802 ALANWOOD

/Users/shubham/anaconda3/envs/hurricane/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [114]:
damage_data.head()

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Orig_RE_Verified_Loss,coordinates
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,58999.0,"27 50m 4.89676s N, 97 4m 15.0212s W"
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,2563.0,"27 48m 6.8256s N, 97 28m 43.4028s W"
7,Household,1202 MESQUITE ST SPACE 546,FULTON,TX,78358.0,26823.0,None
9,Household,314 DRISCOLL ST,WOODSBORO,TX,78393.0,99357.0,"28 14m 4.31942s N, 97 19m 54.288s W"
13,Household,3301 MEADOWLANE ST,VICTORIA,TX,77901.0,55813.0,None


In [100]:
damage_data.coordinates.describe()

TypeError: unhashable type: 'Point'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1653, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'Point'


count                                     4080
unique                                    4001
top       27 54m 28.0141s N, 97 11m 22.5085s W
freq                                         5
Name: coordinates, dtype: object

In [80]:
len(damage_data)

53565

In [94]:
small = damage_data.head(3)

In [95]:
small['coordinates'] = small.apply(lambda x: geocode(x), axis=1)

314 CUT OFF ROAD, PORT ARANSAS, TX, 78373
5901 HORIZON DRIVE, CORPUS CHRISTI, TX, 78408
1202 MESQUITE ST SPACE 546, FULTON, TX, 78358


/Users/shubham/anaconda3/envs/hurricane/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [91]:
small

,applicant_type,Individual_Address,Individual_City,Individual_State,Individual_Zip,Orig_RE_Verified_Loss,coordinates
0,Household,314 CUT OFF ROAD,PORT ARANSAS,TX,78373.0,58999.0,"27 50m 4.89676s N, 97 4m 15.0212s W"
2,Household,5901 HORIZON DRIVE,CORPUS CHRISTI,TX,78408.0,2563.0,"27 48m 6.8256s N, 97 28m 43.4028s W"
7,Household,1202 MESQUITE ST SPACE 546,FULTON,TX,78358.0,26823.0,None
